In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

new_application_df = application_df.drop(["EIN", "NAME", "CLASSIFICATION","APPLICATION_TYPE" ], axis=1)
new_application_df[:5]

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,1,0,N,5000,1
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Convert categorical data to numeric with `pd.get_dummies`
coverted_df= pd.get_dummies(application_df)

In [4]:
# Split our preprocessed data into our features and target arrays
#from sklearn.model_selection import train_test_split
sample_size = int(len(coverted_df) * 0.5)  # 50% of the original dataset as I run out of RAM
sample_df = coverted_df[:sample_size]


# Split the preprocessed data into features and target arrays
X = sample_df.drop('IS_SUCCESSFUL', axis=1).values
y = sample_df['IS_SUCCESSFUL'].values


# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(80, input_dim=X_train.shape[1], activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_scaled, y_train, epochs=10)

# Scale the test data
X_test_scaled = X_scaler.transform(X_test)

# Evaluate the model on the scaled test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)


Epoch 1/10
268/268 [==============================] - 7s 24ms/step - loss: 0.5841 - accuracy: 0.6871
Epoch 2/10
268/268 [==============================] - 7s 25ms/step - loss: 0.3103 - accuracy: 0.8778
Epoch 3/10
268/268 [==============================] - 6s 23ms/step - loss: 0.1063 - accuracy: 0.9668
Epoch 4/10
268/268 [==============================] - 6s 23ms/step - loss: 0.0875 - accuracy: 0.9696
Epoch 5/10
268/268 [==============================] - 5s 20ms/step - loss: 0.0824 - accuracy: 0.9714
Epoch 6/10
268/268 [==============================] - 6s 23ms/step - loss: 0.0807 - accuracy: 0.9707
Epoch 7/10
268/268 [==============================] - 6s 21ms/step - loss: 0.0773 - accuracy: 0.9705
Epoch 8/10
268/268 [==============================] - 7s 24ms/step - loss: 0.0765 - accuracy: 0.9706
Epoch 9/10
268/268 [==============================] - 5s 20ms/step - loss: 0.0737 - accuracy: 0.9714
Epoch 10/10
268/268 [==============================] - 1s 4ms/step - loss: 0.4962 - accurac

In [7]:
# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1574880   
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,577,341
Trainable params: 1,577,341
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Export our model to HDF5 file
model.save("..//deep-learning-challenge/AlphabetSoupCharity_Optimization1.h5")
